## Oleh:
- Putu Gde Kenzie Carlen Mataram (71230994)
<br>
- Tara Tirtanata (71231056)

In [83]:
%pip install backtesting yfinance pandas_ta plotly scikit-learn numpy pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [84]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta
from sklearn.ensemble import RandomForestClassifier
from backtesting import Backtest, Strategy
import warnings

# Konfigurasi Tampilan
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')

In [85]:
# ================= CONFIGURATION =================
# 1. Daftar Aset untuk Diadu
TICKERS = ['QQQ', 'NVDA', 'TSLA', 'AAPL', 'AMZN', 'MSFT', 'GOOGL', 'NFLX'] 

# 2. Timeframe Eksekusi untuk Diuji
# M1 = Sangat agresif (Noise tinggi)
# M5 = Balance
# M15 = Lebih stabil (Sedikit sinyal tapi akurat)
TIMEFRAMES = ['1m', '5m', '15m', '30m']

# 3. Parameter Keamanan (Safety)
ADX_THRESHOLD = 25       # Minimum kekuatan trend (di bawah ini = DILARANG TRADING)
RISK_PER_TRADE = 0.02    # Risiko 2% per entry awal
LEVERAGE = 1.0           # Kita set 1.0 (Tanpa Leverage) dulu untuk melihat performa murni
CASH = 10000             # Modal Awal

print(f"Konfigurasi Siap: Menguji {len(TICKERS)} Aset pada {len(TIMEFRAMES)} Timeframe.")

Konfigurasi Siap: Menguji 8 Aset pada 4 Timeframe.


In [86]:
def get_data(symbol, interval):
    """
    Mengambil data sesuai batasan Yahoo Finance.
    - 1m: Max 7 hari
    - 5m, 15m: Max 60 hari
    """
    period = "7d" if interval == "1m" else "60d"
    
    try:
        df = yf.download(symbol, period=period, interval=interval, progress=False)
        if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
        df = df.rename(columns={'Adj Close': 'Close'})
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
        df = df[df['Volume'] > 0]
        
        # Standarisasi Timezone ke UTC agar cocok saat mapping
        if df.index.tz is None: 
            df.index = df.index.tz_localize('UTC')
        else:
            df.index = df.index.tz_convert('UTC')
            
        return df
    except Exception as e:
        print(f"Gagal download {symbol} ({interval}): {e}")
        return None

def add_technical_features(df):
    df = df.copy()
    # Trend Filter (PENTING UNTUK KEAMANAN)
    adx = ta.adx(df['High'], df['Low'], df['Close'], length=14)
    df['ADX'] = adx[adx.columns[0]] # Mengambil kolom ADX utama
    
    # Trend Direction
    df['EMA_50'] = ta.ema(df['Close'], length=50)
    df['EMA_200'] = ta.ema(df['Close'], length=200)
    
    # Volatility untuk Stop Loss
    df['ATR'] = ta.atr(df['High'], df['Low'], df['Close'], length=14)
    
    # Momentum
    df['RSI'] = ta.rsi(df['Close'], length=14)
    
    # Bollinger Bands (Untuk ML features)
    bb = ta.bbands(df['Close'], length=20, std=2)
    df['BB_Width'] = bb[bb.columns[2]] # Bandwidth
    
    return df.dropna()

In [87]:
# CELL 4: Brain Engine (Machine Learning) - REVISED (ANTI-ERROR)

def train_macro_model(symbol):
    """
    Melatih model Random Forest pada data Harian/Jam 
    untuk menentukan bias jangka panjang (Bullish/Bearish).
    """
    print(f"   [ML] Downloading data training untuk {symbol}...")
    # Download data 1 tahun (max period)
    df = yf.download(symbol, period="1y", interval="1h", progress=False)
    
    # Cleaning MultiIndex column issues from yfinance
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
    
    # Pastikan nama kolom benar
    df = df.rename(columns={'Adj Close': 'Close'})
    
    # Feature Engineering Sederhana
    df['RSI'] = ta.rsi(df['Close'], length=14)
    df['Return'] = df['Close'].pct_change()
    
    # Hati-hati dengan Volume change, jika volume 0 bisa jadi inf
    # Kita tambahkan 1e-9 (angka sangat kecil) untuk menghindari pembagian dengan nol
    df['Vol_Change'] = (df['Volume'] + 1e-9).pct_change()
    
    # --- FIX ERROR: CLEANING INFINITY & NAN ---
    # Mengganti nilai inf/-inf menjadi NaN, lalu didrop
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    # ------------------------------------------
    
    # Validasi Data Kosong setelah cleaning
    if len(df) < 50:
        raise ValueError(f"Data tidak cukup setelah cleaning ({len(df)} rows)")

    # Target: Apakah harga Close besok lebih tinggi?
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    
    features = ['RSI', 'Return', 'Vol_Change']
    X = df[features]
    y = df['Target']
    
    # Double check cleaning (Safety Net terakhir)
    X = X.replace([np.inf, -np.inf], 0).fillna(0)
    
    # Training
    model = RandomForestClassifier(n_estimators=100, min_samples_split=10, random_state=42)
    model.fit(X, y)
    
    # Kembalikan DataFrame dengan sinyal prediksi
    df['Macro_Signal'] = model.predict(X)
    
    if df.index.tz is None: df.index = df.index.tz_localize('UTC')
    else: df.index = df.index.tz_convert('UTC')
        
    return df[['Macro_Signal']]

In [88]:
class SmartPyramid(Strategy):
    # Parameter Optimal (Bisa di-tweak)
    atr_sl_multiplier = 2.0   # Stop Loss jarak 2x ATR
    atr_tp_multiplier = 4.0   # Take Profit (Opsional, kita pakai trailing)
    max_layers = 4            # Maksimal 4 lapis posisi
    adx_filter = 20       # JANGAN TRADE jika ADX < 25 (Pasar Sideways)
    
    def init(self):
        self.highest_price = 0
        self.stop_price = 0
    
    def next(self):
        price = self.data.Close[-1]
        atr = self.data.ATR[-1]
        adx = self.data.ADX[-1]
        macro_signal = self.data.Macro_Signal[-1] # Sinyal dari H1
        ema_50 = self.data.EMA_50[-1]
        
        # --- 1. EXIT LOGIC (Trailing Stop) ---
        if self.position.is_long:
            # Update Trailing Stop jika harga naik
            if price > self.highest_price:
                self.highest_price = price
                # Stop loss naik mengikuti harga (Trailing)
                new_stop = self.highest_price - (atr * self.atr_sl_multiplier)
                self.stop_price = max(self.stop_price, new_stop)
            
            # Eksekusi Exit
            if price < self.stop_price:
                self.position.close()
            
            # Hard Exit jika tren berbalik dibawah EMA 50
            if price < ema_50:
                self.position.close()

        # --- 2. ENTRY LOGIC (Hanya jika Posisi < Max Layers) ---
        # SAFETY CHECK: 
        # 1. Market harus Trending (ADX > 25)
        # 2. Macro (H1) harus Bullish
        # 3. Harga di atas EMA 50 (Uptrend)
        
        is_trending = adx > self.adx_filter
        is_uptrend = price > ema_50
        is_macro_bullish = macro_signal == 1
        
        if is_trending and is_uptrend and is_macro_bullish:
            
            # Entry Pertama
            if not self.position:
                self.buy(size=0.30) # Masuk 30% Equity
                self.highest_price = price
                self.stop_price = price - (atr * self.atr_sl_multiplier)
            
            # Pyramiding (Tambah Muatan)
            elif len(self.trades) < self.max_layers:
                last_entry = self.trades[-1].entry_price
                
                # Hanya nambah jika posisi sebelumnya sudah profit (Scale In Winners)
                if price > last_entry + (1.0 * atr):
                    self.buy(size=0.20) # Tambah 20%

In [89]:
# ==============================================================================
# SCRIPT: EKSEKUSI & EXPORT KE EXCEL
# ==============================================================================

# List untuk menampung seluruh data laporan
all_reports = []

print("===================================================================")
print("              MULAI PROSES KOMPARASI MULTI-ASET & TF")
print("===================================================================\n")

for ticker in TICKERS:
    # ---------------------------------------------
    # 1. Siapkan Otak (Macro Model - H1)
    # ---------------------------------------------
    print(f"--- Melatih Model Macro untuk {ticker} ---")
    try:
        macro_df = train_macro_model(ticker)
    except Exception as e:
        print(f"Skip {ticker}: Gagal train model ({e})")
        continue

    for tf in TIMEFRAMES:
        print(f"Processing: {ticker} pada Timeframe {tf}...", end=" ") # end=" " agar print menyamping
        
        # ---------------------------------------------
        # 2. Siapkan Data Eksekusi (Micro)
        # ---------------------------------------------
        micro_df = get_data(ticker, tf)
        if micro_df is None or len(micro_df) < 50:
            print(f"-> GAGAL (Data tidak cukup).")
            continue
            
        micro_df = add_technical_features(micro_df)
        
        # ---------------------------------------------
        # 3. Gabungkan Sinyal Macro ke Micro
        # ---------------------------------------------
        # Forward fill sinyal H1 ke data menit
        micro_df['Macro_Signal'] = macro_df['Macro_Signal'].reindex(micro_df.index, method='ffill')
        micro_df.dropna(inplace=True)
        
        # ---------------------------------------------
        # 4. Jalankan Backtest
        # ---------------------------------------------
        try:
            # Margin 1.0 = Tanpa Leverage (untuk tes murni)
            bt = Backtest(micro_df, SmartPyramid, cash=CASH, commission=0.0005, margin=1.0)
            stats = bt.run()
            
            # ---------------------------------------------
            # 5. BERSIHKAN & SIMPAN DATA (TIDAK DI-PRINT)
            # ---------------------------------------------
            # Ubah objek stats menjadi Dictionary
            stats_dict = stats.to_dict()
            
            # Hapus objek internal yang tidak bisa masuk Excel (seperti grafik/tabel trade detail)
            # Kita hanya ambil angka-angka statistiknya
            clean_stats = {k: v for k, v in stats_dict.items() if not str(k).startswith('_')}
            
            # Tambahkan identitas Aset & Timeframe di kolom paling depan
            clean_stats['Ticker'] = ticker
            clean_stats['Timeframe'] = tf
            
            # Masukkan ke list utama
            all_reports.append(clean_stats)
            
            print(f"-> SELESAI. (Ret: {stats['Return [%]']:.2f}%)")
            
        except Exception as e:
            print(f"\n   -> Error Backtest {ticker} {tf}: {e}")

print("\n===================================================================")
print("              PROSES SELESAI - MENYIMPAN KE EXCEL")
print("===================================================================")

if len(all_reports) > 0:
    # Buat DataFrame dari list laporan
    df_final = pd.DataFrame(all_reports)
    
    # --- PERBAIKAN ERROR TIMEZONE & DURASI ---
    for col in df_final.columns:
        # 1. Cek jika kolom adalah Datetime (Tanggal/Jam)
        if pd.api.types.is_datetime64_any_dtype(df_final[col]):
            # Hapus informasi Timezone agar diterima Excel
            try:
                df_final[col] = df_final[col].dt.tz_localize(None)
            except:
                pass # Jika sudah polos, biarkan
                
        # 2. Cek jika kolom adalah Timedelta (Durasi)
        # Excel suka error baca durasi, jadi kita ubah ke Teks (String) saja
        elif pd.api.types.is_timedelta64_dtype(df_final[col]):
            df_final[col] = df_final[col].astype(str)
    # -----------------------------------------

    # Rapikan urutan kolom: Ticker & Timeframe taruh paling kiri
    cols = ['Ticker', 'Timeframe'] + [c for c in df_final.columns if c not in ['Ticker', 'Timeframe']]
    df_final = df_final[cols]
    
    # Urutkan berdasarkan Return Tertinggi
    df_final = df_final.sort_values(by='Return [%]', ascending=False)
    
    # Simpan ke Excel
    file_name = 'Laporan_Backtest_Lengkap.xlsx'
    df_final.to_excel(file_name, index=False)
    
    print(f"✅ SUKSES! Laporan lengkap telah disimpan di file: {file_name}")
    print("   Silakan download file tersebut dan buka di Excel.")
    
    # Preview Singkat
    print("\nTOP 5 PERFORMA TERBAIK:")
    # Kita ambil kolom penting saja untuk preview
    preview_cols = ['Ticker', 'Timeframe', 'Return [%]', 'Win Rate [%]', 'Profit Factor']
    # Pastikan kolom ada sebelum di-display (jaga-jaga)
    cols_to_show = [c for c in preview_cols if c in df_final.columns]
    display(df_final[cols_to_show].head(5))

else:
    print("❌ Tidak ada hasil backtest yang berhasil dijalankan.")

              MULAI PROSES KOMPARASI MULTI-ASET & TF

--- Melatih Model Macro untuk QQQ ---
   [ML] Downloading data training untuk QQQ...
Processing: QQQ pada Timeframe 1m... 

Backtest.run:   0%|          | 0/2482 [00:00<?, ?bar/s]

-> SELESAI. (Ret: -1.10%)
Processing: QQQ pada Timeframe 5m... 

Backtest.run:   0%|          | 0/4402 [00:00<?, ?bar/s]

-> SELESAI. (Ret: 2.09%)
Processing: QQQ pada Timeframe 15m... 

...

Backtest.run:   0%|          | 0/1348 [00:00<?, ?bar/s]

-> SELESAI. (Ret: -4.28%)
Processing: NFLX pada Timeframe 30m... 

Backtest.run:   0%|          | 0/574 [00:00<?, ?bar/s]

-> SELESAI. (Ret: -4.88%)

              PROSES SELESAI - MENYIMPAN KE EXCEL
✅ SUKSES! Laporan lengkap telah disimpan di file: Laporan_Backtest_Lengkap.xlsx
   Silakan download file tersebut dan buka di Excel.

TOP 5 PERFORMA TERBAIK:


,Ticker,Timeframe,Return [%],Win Rate [%],Profit Factor
9,TSLA,5m,7.864769,45.517241,1.695146
5,NVDA,5m,6.265993,33.913043,1.899482
7,NVDA,30m,5.431698,38.888889,3.418582
6,NVDA,15m,3.518863,21.875000,2.300703
8,TSLA,1m,3.035714,28.985507,2.536122


In [90]:
# Membuat DataFrame Ranking
df_result = pd.DataFrame(summary_data)

# Urutkan berdasarkan Return Tertinggi (Bisa diubah ke Sharpe jika mau yang paling stabil)
df_ranked = df_result.sort_values(by='Return [%]', ascending=False).reset_index(drop=True)

print("\n===================================================================")
print("                 KESIMPULAN & ANALISIS PAKAR")
print("===================================================================")
print("\nTOP PERFORMA (Berdasarkan Return):")
display(df_ranked)

# Logika Rekomendasi
best_performer = df_ranked.iloc[0]
safest_performer = df_result.sort_values(by='Max Drawdown [%]', ascending=False).iloc[0] # Drawdown paling mendekati 0 (paling tidak minus)

print("\n--- ANALISIS METODE ---")
print(f"1. JUARA PROFIT: {best_performer['Ticker']} di Timeframe {best_performer['Timeframe']}")
print(f"   Alasan: Menghasilkan Return {best_performer['Return [%]']:.2f}% dengan Winrate {best_performer['Win Rate [%]']:.2f}%.")
print(f"   Karakteristik: Cocok untuk metode agresif ini karena volatilitasnya mendukung pyramiding.")

print(f"\n2. JUARA KEAMANAN (Safe Haven): {safest_performer['Ticker']} di Timeframe {safest_performer['Timeframe']}")
print(f"   Alasan: Max Drawdown hanya {safest_performer['Max Drawdown [%]']:.2f}%.")
print(f"   Catatan: Jika Anda ingin tidur nyenyak, gunakan aset dan timeframe ini.")

print("\n--- REKOMENDASI TIME FRAME ---")
avg_return_m1 = df_result[df_result['Timeframe'] == '1m']['Return [%]'].mean()
avg_return_m5 = df_result[df_result['Timeframe'] == '5m']['Return [%]'].mean()
avg_return_m15 = df_result[df_result['Timeframe'] == '15m']['Return [%]'].mean()

print(f"Rata-rata Return M1 : {avg_return_m1:.2f}% (Hanya 7 hari data)")
print(f"Rata-rata Return M5 : {avg_return_m5:.2f}% (60 hari data)")
print(f"Rata-rata Return M15: {avg_return_m15:.2f}% (60 hari data)")

if avg_return_m1 > avg_return_m5:
    print("-> Timeframe M1 terlihat lebih menguntungkan dalam jangka pendek, namun hati-hati noise.")
else:
    print("-> Timeframe M5/M15 lebih stabil dan konsisten untuk strategi trend following.")


                 KESIMPULAN & ANALISIS PAKAR

TOP PERFORMA (Berdasarkan Return):


,Ticker,Timeframe,Return [%],Win Rate [%],Sharpe Ratio,Max Drawdown [%],# Trades,Profit Factor,Duration
0,TSLA,5m,7.864769,45.517241,3.140205,-4.067048,145,1.695146,82 days 03:45:00
1,NVDA,5m,6.265993,33.913043,2.221944,-2.468553,115,1.899482,82 days 03:50:00
2,AMD,5m,5.271298,45.689655,1.810028,-3.512797,116,1.378156,82 days 03:40:00
3,NVDA,15m,3.518863,21.875000,1.420617,-4.466159,32,2.300703,75 days 03:00:00
4,TSLA,1m,3.035714,28.985507,5.165923,-1.961018,69,2.536122,8 days 03:05:00
5,AMZN,5m,2.842180,41.463415,2.215989,-1.671282,123,1.744749,82 days 03:45:00
6,AAPL,15m,2.429943,44.680851,1.558137,-3.185572,47,2.262334,75 days 03:00:00
7,GOOGL,15m,2.262463,35.849057,1.100090,-3.668948,53,1.350744,75 days 03:00:00
8,QQQ,5m,2.091149,44.262295,2.299225,-1.331739,122,1.792686,82 days 03:45:00
9,AMZN,15m,1.922100,39.215686,1.487135,-1.585625,51,1.406896,75 days 03:00:00



--- ANALISIS METODE ---
1. JUARA PROFIT: TSLA di Timeframe 5m
   Alasan: Menghasilkan Return 7.86% dengan Winrate 45.52%.
   Karakteristik: Cocok untuk metode agresif ini karena volatilitasnya mendukung pyramiding.

2. JUARA KEAMANAN (Safe Haven): GC=F di Timeframe 1m
   Alasan: Max Drawdown hanya -0.00%.
   Catatan: Jika Anda ingin tidur nyenyak, gunakan aset dan timeframe ini.

--- REKOMENDASI TIME FRAME ---
Rata-rata Return M1 : -0.81% (Hanya 7 hari data)
Rata-rata Return M5 : 2.26% (60 hari data)
Rata-rata Return M15: 1.17% (60 hari data)
-> Timeframe M5/M15 lebih stabil dan konsisten untuk strategi trend following.


In [ ]:
# ==============================================================================
# SCRIPT: UPDATE DATA M30 & VISUALISASI 4 GRAFIK (1m, 5m, 15m, 30m)
# ==============================================================================
import plotly.graph_objects as go
import plotly.express as px

# 1. KONFIGURASI BARU
# Kita tambah '30m' ke dalam daftar
TIMEFRAMES_LENGKAP = ['1m', '5m', '15m', '30m']
MIN_TRADES = 5 

print("===================================================================")
print("        UPDATE DATA BACKTEST (MENAMBAHKAN M30)")
print("===================================================================")

# Cek apakah df_final sudah ada, jika belum kita buat baru
if 'df_final' not in locals():
    # Jika user belum pernah run sebelumnya, kita init list baru (tapi idealnya run code backtest utama dulu)
    print("⚠ Peringatan: df_final belum ada. Kode ini akan mencoba menjalankan backtest parsial.")
    all_reports = [] 
else:
    # Kita ambil data yang sudah ada di memori
    all_reports = df_final.to_dict('records')

# Kita perlu memastikan data M30 masuk ke df_final
# Loop cek: Apakah '30m' sudah ada di laporan? Jika belum, kita run backtest-nya.
existing_tfs = set([r['Timeframe'] for r in all_reports if 'Timeframe' in r])

if '30m' not in existing_tfs:
    print("⏳ Sedang menghitung data untuk Timeframe 30m...")
    for ticker in TICKERS:
        try:
            # Setup Data & Model
            df = get_data(ticker, '30m')
            if df is None: continue
            df = add_technical_features(df)
            
            macro_df = train_macro_model(ticker)
            df['Macro_Signal'] = macro_df['Macro_Signal'].reindex(df.index, method='ffill')
            df.dropna(inplace=True)
            
            # Run Backtest
            bt = Backtest(df, SmartPyramid, cash=CASH, commission=0.0005, margin=LEVERAGE)
            stats = bt.run()
            
            # Simpan Hasil
            stats_dict = stats.to_dict()
            clean_stats = {k: v for k, v in stats_dict.items() if not str(k).startswith('_')}
            clean_stats['Ticker'] = ticker
            clean_stats['Timeframe'] = '30m'
            
            all_reports.append(clean_stats)
            print(f"   -> {ticker} [30m]: Return {stats['Return [%]']:.2f}%")
            
        except Exception as e:
            print(f"   -> Gagal {ticker}: {e}")
            
    # Update DataFrame df_final dengan data baru
    df_final = pd.DataFrame(all_reports)
    print("✅ Data M30 berhasil ditambahkan ke df_final.")
else:
    print("ℹ️ Data M30 sudah tersedia, lanjut ke visualisasi.")


# ==============================================================================
# BAGIAN VISUALISASI (4 GRAFIK)
# ==============================================================================

for tf in TIMEFRAMES_LENGKAP:
    print(f"\n" + "-"*60)
    print(f"  MEMBUAT GRAFIK: TIMEFRAME {tf}")
    print("-"*60)

    # A. Filter Top 5
    subset = df_final[df_final['Timeframe'] == tf]
    if subset.empty:
        print(f"⚠ Skip {tf}: Tidak ada data.")
        continue
        
    qualified = subset[subset['# Trades'] >= MIN_TRADES]
    if qualified.empty: qualified = subset # Fallback

    top_performers = qualified.sort_values(by='Return [%]', ascending=False).head(5)
    assets_to_plot = top_performers['Ticker'].tolist()
    
    if not assets_to_plot:
        print(f"⚠ Tidak ada aset valid di {tf}.")
        continue

    print(f"⭐ JUARA {tf}: {assets_to_plot}")
    
    # B. Ambil Kurva Ekuitas
    equity_curves = {}
    color_cycle = px.colors.qualitative.Bold 
    color_map_manual = {'QQQ': '#00FF00', 'NVDA': '#FF4500', 'GC=F': '#C0C0C0', 'BTC-USD': '#F7931A', 'SPY': '#00BFFF'}

    for i, ticker in enumerate(assets_to_plot):
        try:
            df = get_data(ticker, tf)
            if df is None: continue
            df = add_technical_features(df)
            macro = train_macro_model(ticker)
            df['Macro_Signal'] = macro['Macro_Signal'].reindex(df.index, method='ffill')
            df.dropna(inplace=True)
            
            bt = Backtest(df, SmartPyramid, cash=CASH, commission=0.0005, margin=LEVERAGE)
            stats = bt.run()
            equity_curves[ticker] = stats._equity_curve['Equity']
        except:
            pass

    # C. Plotting
    if equity_curves:
        fig = go.Figure()
        for i, (ticker, curve) in enumerate(equity_curves.items()):
            start_equity = curve.iloc[0]
            norm_curve = (curve - start_equity) / start_equity * 100
            c = color_map_manual.get(ticker, color_cycle[i % len(color_cycle)])
            
            fig.add_trace(go.Scatter(
                x=norm_curve.index, y=norm_curve, mode='lines', name=ticker,
                line=dict(width=2, color=c)
            ))

        fig.update_layout(
            title=f'<b>TOP 5 PERFORMERS - {tf}</b><br><sup>(Min {MIN_TRADES} Trades)</sup>',
            xaxis_title='Waktu', yaxis_title='Profit (%)',
            template='plotly_dark', hovermode='x unified', height=450,
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01)
        )
        fig.show()

        UPDATE DATA BACKTEST (MENAMBAHKAN M30)
ℹ️ Data M30 sudah tersedia, lanjut ke visualisasi.

------------------------------------------------------------
  MEMBUAT GRAFIK: TIMEFRAME 1m
------------------------------------------------------------
⭐ JUARA 1m: ['TSLA', 'AMZN', 'AAPL', 'GOOGL', 'QQQ']
   [ML] Downloading data training untuk TSLA...


Backtest.run:   0%|          | 0/2509 [00:00<?, ?bar/s]

   [ML] Downloading data training untuk AMZN...


Backtest.run:   0%|          | 0/2522 [00:00<?, ?bar/s]

   [ML] Downloading data training untuk AAPL...


...

   [ML] Downloading data training untuk GOOGL...


Backtest.run:   0%|          | 0/574 [00:00<?, ?bar/s]

   [ML] Downloading data training untuk QQQ...


Backtest.run:   0%|          | 0/574 [00:00<?, ?bar/s]


✅ Selesai! Sekarang Anda punya 4 perbandingan timeframe.
